# Find Subgame-Perfect Equilibria for 2 periods, 2 teams of 2 riders

Functions and imports

In [3]:
import numpy as np
import itertools
from itertools import combinations_with_replacement as cr
from matplotlib import pyplot as plt
from fractions import Fraction
import pygambit as pg


def update(s, E, position): 
    """
    Args:
        s (np.array): actions selected by the players
            0: Rest
            1: Pace
            1.4: Mark
            2: Attack
        E (np.array[int]): energy levels of the players
        position (np.array[int]): position of the players

        assert s.shape == E.shape == position.shape
    
    Returns:
        tuple(array, array): new energy
    """
    #works for arbitrary number of teams, riders
    shape = np.shape(s)#team structure of the race
    pos = position.flatten()
    ind = np.unique(pos)
    add = np.zeros(np.shape(pos))
    for j in ind:
        temp = np.zeros(np.shape(pos))
        temp[pos == j] = s.flatten()[pos==j] #strategies of the current group
        if 2 in temp:
            temp[temp == 1.4] = 2
        else:
            temp[temp == 1.4] = 0 #order matters: the 1.4 guy moves at 1 if there is no 2 but a 1, if there is none of either, he moves at 0!
            
        if 1 in temp:
            temp[temp == 0] = 1 #if there is a helper, all 0's move at speed 1

        temp[pos != j] = 0
        add = add + temp

    pos = pos+add
    pos = pos.reshape(shape)
    new = E.flatten() - np.round(s.flatten()) #effort level of 1.4 also actually costs 1
    E = new.reshape(shape) #remove the exerted effort from the batteries
    return E, pos 

def payoff(E, position): #works for arbitrary number of teams, riders
    shape = np.shape(E)
    p = np.zeros(shape[0])
    lead = E.flatten()
    lead[position.flatten() != max(position.flatten())] = 0
    if sum(lead) != 0:
        lead = lead.reshape(shape)
        for ind in range(shape[0]):
            p[ind] = max(lead[ind])
        p = p/sum(p) 
    else:
        lead = position.flatten()
        lead[position.flatten() != max(position.flatten())] = 0
        lead[position.flatten() == max(position.flatten())] = 1
        lead = lead.reshape(shape)  
        for ind in range(shape[0]):
            p[ind] = 1 in lead[ind]
        p = p/sum(p)        
    return p
    

def feas_strategies_ind(E): #for individual rider
    """
    Return all possible combinations that add up to at most E
    in T distinct time periods in all possible ways, respecting order.
    """
    all_part = []
    for index in range(min(int(E)+1, 3)):
        counts = [index]
        all_part.append(counts)
        if index == 1:
            counts= [1.4]
            all_part.append(counts)
    return all_part

def feas_strategy_combs(E): #works for arbitrary number of teams, riders
    shape = np.shape(E) #(team, rider)
    res = []
    for i in range(len(E.flatten())):
        res.append(feas_strategies_ind(E.flatten()[i]))
        
    strats = [[feas_strategies_ind(E[i][j]) for j in range(shape[1])] for i in range(shape[0])] 
    team_strats = []
    for i in range(shape[0]):
        team_strats.append(list(itertools.product(*strats[i])))
        
    for i in range(shape[0]):
        team_strats[i] = np.reshape(team_strats[i], (np.shape(team_strats[i])[0], np.shape(team_strats[i])[1]))

    all_strats = list(itertools.product(*team_strats))
    s = np.array(all_strats)
    
    j = 0
    while (s[j][0].any() == 0):
        j += 1
        if j == len(s):
            break
        
    strats2 = s[0:j,1]
    j_1 = int(len(s)/j)
    strats1 = []
    for i in range(j_1):
        strats1.append(s[j*i,0])
    return all_strats, np.array(strats1), strats2 #riderstrat = strategies(variant, team, rider)

# Subgame Perfect equilibria

In [4]:
def spe2(eff, pos, printing):
    #find all strategies
    strats1 = feas_strategy_combs(eff)[1]
    strats2 = feas_strategy_combs(eff)[2]
    poms = []

    overall_pom = np.zeros([len(strats1),len(strats2)])

    #Find all possible (effort, position)-situations for period 2, so that no equilibrium computations are done multiple times:
    efforts = []
    positions = []
    mat = []

    for i in range(len(strats1)):
        for j in range(len(strats2)):
            temp = np.vectorize(int)(update(np.array([strats1[i], strats2[j]]), eff, pos))
            efforts.append(temp[0])
            positions.append(temp[1])
            mat.append([i,j])

    # Calculate payoff matrix for all given situations, then use Nashpy to find the equilibria. At the end use key-list from above to put player 1's payoffs into the corresponding sports of the payoff matrix.

    for lauf in range(len(efforts)):
        eff = np.array(efforts[lauf])
        pos = np.array(positions[lauf])

        s1 = feas_strategy_combs(eff)[1]
        s2 = feas_strategy_combs(eff)[2]


        pom = np.zeros([len(s1),len(s2)])

        for i in range(len(s1)):
            for j in range(len(s2)):
                pom[i,j]=payoff(update(np.array([s1[i], s2[j]]), eff, pos)[0], update(np.array([s1[i], s2[j]]), eff, pos)[1])[0]
        poms.append(pom)

        m1 = np.vectorize(Fraction.limit_denominator)(np.vectorize(Fraction)(pom))
        m2 = 1 - m1
        g = pg.Game.from_arrays(m1, m2)    
        sol = pg.nash.lp_solve(g)
        c = g.mixed_strategy_profile(sol).payoff(g.players[0])
        overall_pom[mat[lauf][0], mat[lauf][1]] = c


    if printing == 'yes':
        for i in range(len(strats1)):
            for j in range(len(strats2)):
                print(Fraction(overall_pom[i,j]).limit_denominator(), ' ', end='')
            print('\n')

    #find equilibrium strategies of the first period
    pom = overall_pom


    return overall_pom, poms


In [5]:
eff = np.array([[3, 3], [2, 2]]) #needs to be in ascending order 
pos = np.array([[0, 0], [0, 0]]) #here, order doesn't matter

#feas_strategy_combs(eff)[1]


eff


array([[3, 3],
       [2, 2]])

In [6]:



pom = spe2(eff, pos, 'no')[0]
m1 = np.vectorize(Fraction.limit_denominator)(np.vectorize(Fraction)(pom))
m2 = 1 - m1
g = pg.Game.from_arrays(m1, m2)    
sol = pg.nash.enummixed_solve(g)
for i in range(len(sol)):
    print('Solution number', i+1, 'has payoff', float(g.mixed_strategy_profile(sol[i]).payoff(g.players[0])))
    print('And Strategy vectors', np.vectorize(float)(sol[i])[0:16],'\n','\n', np.vectorize(float)(sol[i])[16:])

KeyboardInterrupt: 

In [ ]:
eff = np.array([[3, 3], [2, 2]]) #needs to be in ascending order 
pos = np.array([[0, 0], [0, 0]]) #here, order doesn't matter
pom = spe2(eff, pos, 'no')[0]
m1 = np.vectorize(Fraction.limit_denominator)(np.vectorize(Fraction)(pom))
m2 = 1 - m1
g = pg.Game.from_arrays(m1, m2)    
sol = pg.nash.enummixed_solve(g)
for i in range(len(sol)):
    print('Solution number', i+1, 'has payoff', float(g.mixed_strategy_profile(sol[i]).payoff(g.players[0])))
    print('And Strategy vectors', np.vectorize(float)(sol[i])[0:16],'\n','\n', np.vectorize(float)(sol[i])[16:])

This version is more efficient, but that's apparently not required

In [ ]:
def spe(eff, pos, printing):
    #find all strategies
    strats1 = feas_strategy_combs(eff)[1]
    strats2 = feas_strategy_combs(eff)[2]

    overall_pom = np.zeros([len(strats1),len(strats2)])

    #Find all possible (effort, position)-situations for period 2, so that no equilibrium computations are done multiple times:

    dict = {}
    payoff_dict = {}

    for i in range(len(strats1)):
        for j in range(len(strats2)):
            temp = np.vectorize(int)(update(np.array([strats1[i], strats2[j]]), eff, pos))

            for t in range(len(temp[0])): #sort efforts and corresponding positions (only for 2 teams of 2 riders)
                effi = temp[0]
                posi = temp[1]
                if effi[t][0] > effi[t][1]:
                    temp1 = effi[t][0]
                    temp2 = posi[t][0]
                    effi[t][0] = effi[t][1]
                    effi[t][1] = temp1
                    posi[t][0] = posi[0][1]
                    posi[t][1] = temp2

            temp[1] = temp[1]-min(temp[1].flatten())
            dict[str(strats1[i]) + str(strats2[j])] = str(temp[0][0])+str(temp[0][1])+str(temp[1][0])+str(temp[1][1])
            payoff_dict[str(i)+','+str(j)] = str(temp[0][0])+str(temp[0][1])+str(temp[1][0])+str(temp[1][1])

    inv_dict = {}
    for k, v in dict.items():
        inv_dict[v] = inv_dict.get(v, []) + [k]

    inv_pdict = {}
    for k, v in payoff_dict.items():
        inv_pdict[v] = inv_pdict.get(v, []) + [k]

    payoff_list = {}
    for i in list(inv_pdict.keys()):
        payoff_list[i] = 0

    efforts = []
    positions = []
    for k in list(payoff_list.keys()):
        efforts.append([[int(k[1]), int(k[3])], [int(k[6]), int(k[8])]])
        positions.append([[int(k[11]), int(k[13])], [int(k[16]), int(k[18])]])

    # Calculate payoff matrix for all given situations, then use Nashpy to find the equilibria. At the end use key-list from above to put player 1's payoffs into the corresponding sports of the payoff matrix.

    pay = []
    poms = []
    for lauf in range(len(efforts)):
        eff = np.array(efforts[lauf])
        pos = np.array(positions[lauf])


        s1 = feas_strategy_combs(eff)[1]
        s2 = feas_strategy_combs(eff)[2]

        pom = np.zeros([len(s1),len(s2)])

        for i in range(len(s1)):
            for j in range(len(s2)):
                pom[i,j]=payoff(update(np.array([s1[i], s2[j]]), eff, pos)[0], update(np.array([s1[i], s2[j]]), eff, pos)[1])[0]
                pom[i,j]=Fraction(pom[i,j]).limit_denominator()

        poms.append(pom)

        maxden = max([np.vectorize(Fraction.limit_denominator)(np.vectorize(Fraction)(pom.flatten()))[i].denominator for i in range(len(pom.flatten()))])
        m1 = np.array(np.vectorize(int)(maxden*pom), dtype=pg.Rational)
        m2 = np.array(maxden-np.vectorize(int)(maxden*pom), dtype=pg.Rational)
        g = pg.Game.from_arrays(m1, m2)    
        sol = pg.nash.lp_solve(g)
        c = g.mixed_strategy_profile(sol).payoff(g.players[0])
        pay.append(c/maxden)

    for i in range(len(list(inv_pdict.values()))):
        for k in list(inv_pdict.values())[i]:
            overall_pom[int(k.split(',')[0]), int(k.split(',')[1])] = pay[i]

    if printing == 'yes':
        for i in range(len(strats1)):
            for j in range(len(strats2)):
                print(Fraction(overall_pom[i,j]).limit_denominator(), ' ', end='')
            print('\n')

    #find equilibrium strategies of the first period
    pom = overall_pom
    maxden = max([np.vectorize(Fraction.limit_denominator)(np.vectorize(Fraction)(pom.flatten()))[i].denominator for i in range(len(pom.flatten()))])
    m1 = np.array(np.vectorize(int)(maxden*pom), dtype=pg.Rational)
    m2 = np.array(maxden-np.vectorize(int)(maxden*pom), dtype=pg.Rational)
    g = pg.Game.from_arrays(m1, m2)    
    sol = pg.nash.lp_solve(g)
    eq1 = {}
    eq2 = {}
    #sol = pg.nash.enummixed_solve(g)
    for t in range(len(sol)):
        soli = np.vectorize(float)(sol[t])
        sol1 = soli[0:len(strats1)]
        sol2 = soli[len(strats1):]
        soli = [sol1, sol2]
        payoff_full = g.mixed_strategy_profile(soli).payoff(g.players[0])/maxden
        for i in range(len(sol1)):
            if sol1[i] != 0:
                eq1[str(t)+':'+str(strats1[i])] = sol1[i]
        for i in range(len(sol2)):
            if sol2[i] != 0:
                eq2[str(t)+':'+str(strats2[i])] = sol2[i]
        eq1['payoff_1'+str(t)] = float(payoff_full)
        eq2['payoff_2'+str(t)] = 1 - float(payoff_full)

    return eq1, eq2, np.vectorize(float)(sol)
